In [2]:
import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F

import tensorflow_datasets as tfds

import numpy as np

import matplotlib.pyplot as plt

from torch.utils.data import TensorDataset, DataLoader
from torch import optim 

device = torch.device("cuda:0")

torch.cuda.set_device(device)

# Load Training and Testing Data Sets

In [ ]:
training_images, training_labels = tfds.as_numpy(tfds.load(
                                        'mnist',
                                        split='train', 
                                        batch_size=-1, 
                                        as_supervised=True,
                                    ))

In [ ]:
testing_images, testing_labels = tfds.as_numpy(tfds.load(
                                        'mnist',
                                        split='test', 
                                        batch_size=-1, 
                                        as_supervised=True,
                                    ))

# Create and Train Simple Convolution Neural Network

In [ ]:
model = nn.Sequential(
                nn.Conv1d(1,10,kernel_size=(3,3)),
                nn.ReLU(),
                nn.Conv1d(10,100,kernel_size=(3,3)),
                nn.ReLU(),
                nn.Flatten(),
                nn.Linear(57600,10),
            )

In [ ]:
def TrainNetwork(model,device,training_loader,testing_loader,numIter):

    model.to(device)

    loss_function = nn.CrossEntropyLoss()
    optimizer     = optim.Adam(model.parameters())

    loss_array     = np.zeros([numIter,2])
    accuracy_array = np.zeros([numIter,2])

    for epoch in range(numIter):

        training_loss_array = []
        testing_loss_array  = []

        training_correct = 0
        testing_correct  = 0

        for image_batch, label_batch in training_loader:

            model.train()

            optimizer.zero_grad()
            
            output            = model(image_batch.to(device))
            
            training_loss = loss_function(output,label_batch.to(device))
            
            training_loss.backward()
            optimizer.step()

            training_loss_array.append(training_loss.cpu().item())
            
            _, predicted      = torch.max(output.data, 1)
            training_correct += (predicted.cpu() == label_batch).sum().item()
            
        for image_batch, label_batch in testing_loader:

            model.eval()
            
            output            = model(image_batch.to(device))
            
            testing_loss = loss_function(model(image_batch.to(device)),label_batch.to(device))
            
            testing_loss_array.append(testing_loss.cpu().item())

            _, predicted     = torch.max(output.data, 1)
            testing_correct += (predicted.cpu() == label_batch).sum().item()
            
        print('Epoch: ' + str(epoch))
        print()
        print("Training Loss: " + '\t\t' + str(np.mean(training_loss_array)))
        print("Testing Loss: " + '\t\t' + str(np.mean(testing_loss_array)))
        print("Training Accuracy: " + '\t' + str(100.0*training_correct/len(training_labels)) + '%')
        print("Testing Accuracy: " + '\t' + str(100.0*testing_correct/len(testing_labels)) + '%')

        loss_array[epoch,0] = np.mean(training_loss_array)
        loss_array[epoch,1] = np.mean(testing_loss_array)
        
        accuracy_array[epoch,0] = np.mean(100.0*training_correct/len(training_labels))
        accuracy_array[epoch,1] = np.mean(100.0*testing_correct/len(testing_labels))
        
        print()
        
    return loss_array, accuracy_array


In [ ]:
labels = np.zeros([10,10])

for ii in range(10):
    
    labels[ii,ii] = 1

In [ ]:
training_images_pytorch = torch.Tensor(training_images).transpose(1,3)
training_labels_pytorch = torch.Tensor(training_labels).type(torch.LongTensor)

testing_images_pytorch = torch.Tensor(testing_images).transpose(1,3)
testing_labels_pytorch = torch.Tensor(testing_labels).type(torch.LongTensor)

training_dataset = TensorDataset(training_images_pytorch,training_labels_pytorch)
testing_dataset  = TensorDataset(testing_images_pytorch,testing_labels_pytorch)

training_loader = DataLoader(training_dataset, batch_size=5, shuffle=True, pin_memory=True,drop_last=True)
testing_loader  = DataLoader(testing_dataset, batch_size=5, shuffle=True, pin_memory=True,drop_last=True)

In [ ]:
loss_array, accuracy_array = TrainNetwork(model,device,training_loader,testing_loader,10)

# Functions for Detection using Filtering methods

In [ ]:
def calculate_lambda(labels,label_vals):
    
    lambda_vals = np.zeros([len(label_vals)])
    
    for val in range(len(label_vals)):
        
        lambda_vals[val] = (val == labels).sum().item()/len(labels)
        
    return lambda_vals

def LambdaPredictionTransition(x,lambda_vals):
    
    x = x + lambda_vals
    return x

def LambdaFilterTransition(x,y):
    
    x = x + 0.01*(y-x)
    return x

def LambdaObservataion(y,label_pred):
    
    lambda_vals = calculate_lambda(label_pred,torch.Tensor(np.arange(0,10)).int())
    y = y + lambda_vals
    return y
    
def Residual(x,y):
    
    r = np.linalg.norm(x-y)
    return r


# Data set manipulation for detection tests

Here we remove half of the examples of the MNIST data set that are labeled as number 0. We are to test if there is a shift in the distribution of the detection signal

In [ ]:
num_examples     = len(np.where(testing_labels>0)[0])+int(len(np.where(testing_labels==0)[0])/2)
detection_images = torch.zeros([num_examples,1,28,28])
detection_labels = torch.zeros([num_examples])

image_ids        = np.where(testing_labels>0)
rand_id          = np.random.randint(len(image_ids[0]),size=[int(len(image_ids[0]))])

detection_images[0:len(np.where(testing_labels>0)[0]),:,:,:] = testing_images_pytorch[image_ids[0][rand_id],:,:,:]
detection_labels[0:len(np.where(testing_labels>0)[0])]       = testing_labels_pytorch[image_ids[0][rand_id]]

image_ids        = np.where(testing_labels==0)
rand_id          = np.random.randint(len(image_ids[0]),size=[int(len(image_ids[0])/2)])

detection_images[len(np.where(testing_labels>0)[0]):,:,:,:] = testing_images_pytorch[image_ids[0][rand_id],:,:,:]
detection_labels[len(np.where(testing_labels>0)[0]):]       = testing_labels_pytorch[image_ids[0][rand_id]]


In [ ]:
lambda_vals_training  = calculate_lambda(training_labels_pytorch,torch.Tensor(np.arange(0,10)).int())
lambda_vals_testing   = calculate_lambda(testing_labels_pytorch,torch.Tensor(np.arange(0,10)).int())
lambda_vals_detection = calculate_lambda(detection_labels,torch.Tensor(np.arange(0,10)).int())

# Effect of batch size on detection

Here we increase the batch size to see how this affects the detection variable

In [ ]:
detection_dataset = TensorDataset(detection_images,detection_labels)

no_shift_detection_loader  = DataLoader(testing_dataset, batch_size=1, shuffle=True, pin_memory=True,drop_last=True)
shift_detection_loader     = DataLoader(detection_dataset, batch_size=1, shuffle=True, pin_memory=True,drop_last=True)

In [ ]:
k = 0

x = np.zeros([len(lambda_vals_training),len(no_shift_detection_loader)])
y = np.zeros([len(lambda_vals_training),len(no_shift_detection_loader)])
r = np.zeros([len(no_shift_detection_loader)])

for image_batch, label_batch in no_shift_detection_loader:

    model.eval()

    output           = model(image_batch.to(device))

    _, predicted     = torch.max(output.data, 1)
    
    x[:,k+1] = LambdaPredictionTransition(x[:,k],lambda_vals_training)
    y[:,k+1] = LambdaObservataion(y[:,k],predicted)
    r[k]     = Residual(x[:,k+1],y[:,k+1])
    x[:,k+1] = LambdaFilterTransition(x[:,k+1],y[:,k+1])

    k += 1
    
    if k+1 == len(no_shift_detection_loader):
        break

r_test = r

k = 0

x = np.zeros([len(lambda_vals_training),len(shift_detection_loader)])
y = np.zeros([len(lambda_vals_training),len(shift_detection_loader)])
r = np.zeros([len(shift_detection_loader)])

for image_batch, label_batch in shift_detection_loader:

    model.eval()

    output           = model(image_batch.to(device))

    _, predicted     = torch.max(output.data, 1)
    
    x[:,k+1] = LambdaPredictionTransition(x[:,k],lambda_vals_training)
    y[:,k+1] = LambdaObservataion(y[:,k],predicted)
    r[k]     = Residual(x[:,k+1],y[:,k+1])
    x[:,k+1] = LambdaFilterTransition(x[:,k+1],y[:,k+1])

    k += 1
    
    if k+1 == len(shift_detection_loader):
        break

r_detect = r

In [ ]:
line_up,   = plt.plot(r_test, label='No Label Shift')
line_down, = plt.plot(r_detect, label='Label Shift')
plt.legend(handles=[line_up, line_down])

In [ ]:
bins = np.linspace(0,14,100)

hist_no_shift, _ = np.histogram(r_test, bins=bins, density=True)
hist_shift, _    = np.histogram(r_detect, bins=bins, density=True)

line_up,   = plt.plot(bins[0:99],hist_no_shift, label='No Label Shift')
line_down, = plt.plot(bins[0:99],hist_shift, label='Label Shift')

plt.legend(handles=[line_up, line_down])

In [ ]:
detection_dataset = TensorDataset(detection_images,detection_labels)

no_shift_detection_loader  = DataLoader(testing_dataset, batch_size=5, shuffle=True, pin_memory=True,drop_last=True)
shift_detection_loader     = DataLoader(detection_dataset, batch_size=5, shuffle=True, pin_memory=True,drop_last=True)

In [ ]:
k = 0

x = np.zeros([len(lambda_vals_training),len(no_shift_detection_loader)])
y = np.zeros([len(lambda_vals_training),len(no_shift_detection_loader)])
r = np.zeros([len(no_shift_detection_loader)])

for image_batch, label_batch in no_shift_detection_loader:

    model.eval()

    output           = model(image_batch.to(device))

    _, predicted     = torch.max(output.data, 1)
    
    x[:,k+1] = LambdaPredictionTransition(x[:,k],lambda_vals_training)
    y[:,k+1] = LambdaObservataion(y[:,k],predicted)
    r[k]     = Residual(x[:,k+1],y[:,k+1])
    x[:,k+1] = LambdaFilterTransition(x[:,k+1],y[:,k+1])

    k += 1
    
    if k+1 == len(no_shift_detection_loader):
        break

r_test = r

k = 0

x = np.zeros([len(lambda_vals_training),len(shift_detection_loader)])
y = np.zeros([len(lambda_vals_training),len(shift_detection_loader)])
r = np.zeros([len(shift_detection_loader)])

for image_batch, label_batch in shift_detection_loader:

    model.eval()

    output           = model(image_batch.to(device))

    _, predicted     = torch.max(output.data, 1)
    
    x[:,k+1] = LambdaPredictionTransition(x[:,k],lambda_vals_training)
    y[:,k+1] = LambdaObservataion(y[:,k],predicted)
    r[k]     = Residual(x[:,k+1],y[:,k+1])
    x[:,k+1] = LambdaFilterTransition(x[:,k+1],y[:,k+1])

    k += 1
    
    if k+1 == len(shift_detection_loader):
        break

r_detect = r

In [ ]:
line_up,   = plt.plot(r_test, label='No Label Shift')
line_down, = plt.plot(r_detect, label='Label Shift')
plt.legend(handles=[line_up, line_down])

In [ ]:
bins = np.linspace(0,14,100)

hist_no_shift, _ = np.histogram(r_test, bins=bins, density=True)
hist_shift, _    = np.histogram(r_detect, bins=bins, density=True)

line_up,   = plt.plot(bins[0:99],hist_no_shift, label='No Label Shift')
line_down, = plt.plot(bins[0:99],hist_shift, label='Label Shift')

plt.legend(handles=[line_up, line_down])